In [112]:
#imports and scraping raw data from site
#df is the actual betting line
#df2 has other info, mostly using to get player id to name dict

import pandas as pd
import datetime as dt
import re
from unidecode import unidecode
from betting_functions import get_url_json

In [113]:
dog_props_url = 'https://api.underdogfantasy.com/beta/v3/over_under_lines'
data = get_url_json(dog_props_url)

In [114]:
data.keys()

dict_keys(['appearances', 'games', 'over_under_lines', 'players', 'solo_games'])

In [115]:
df = pd.json_normalize(list(data.values()))
df.index = data.keys()
new = df.T['over_under_lines'].values
lines = pd.json_normalize(new)
lines.sample(2)

,id,live_event,live_event_stat,options,over_under_id,rank,stat_value,status,over_under.id,over_under.appearance_stat.id,over_under.appearance_stat.appearance_id,over_under.appearance_stat.display_stat,over_under.appearance_stat.graded_by,over_under.appearance_stat.pickem_stat_id,over_under.appearance_stat.stat,over_under.boost,over_under.scoring_type_id,over_under.title
3055,bd2e9c87-4bc9-4f5e-a7ec-d80c34b491b0,False,None,[{'id': 'fded897f-700e-46d4-8709-3c7bf5cf9bb9'...,23d649b1-0cf3-422a-9988-0dd836dfa581,25190053,2.0,active,23d649b1-0cf3-422a-9988-0dd836dfa581,178affe2-cbab-4fdb-a810-a9d1f79b79d4,ce0d15b3-1fd3-48e7-b923-4a0b826e6afe,Kills in Maps 1+2+3,high_score,58928a02-7fb5-4f3e-a004-74a205514a66,kills_in_maps_1_2_3,None,None,LoL: Garank Kills in Maps 1+2+3 O/U
2952,e786ba23-b570-41fd-9b4c-6072bcbc4035,False,None,[{'id': '81121dbb-1fbe-4597-98b9-a089f6f0db87'...,58f751be-029b-4b75-ae80-050b608a347d,25189129,5.0,active,58f751be-029b-4b75-ae80-050b608a347d,6b918670-fdea-4907-991f-ab7eac88b635,d6bc0db0-7ea5-4f72-ad6e-61a8fcd22edf,Assists on Map 2,high_score,da6d78ce-4837-4bd6-9257-8bde0eb93c74,assists_in_map_2,None,None,LoL: Flawed Assists on Map 2 O/U


In [116]:
test =  pd.json_normalize(df.T['players'])
player_df = test.copy(deep=True)
player_df = player_df[['id','first_name', 'last_name',
       'sport_id','team_id']]
player_df = player_df.dropna()#

player_df.loc[player_df.sport_id=='ESPORTS']
x = player_df[player_df.sport_id=='ESPORTS']
p2esport = x[['last_name','first_name']].to_dict('split')
p2esport = dict(p2esport['data'])

In [117]:
test =  pd.json_normalize(df.T['players'])
player_df = test.copy(deep=True)
player_df = player_df[['id','first_name', 'last_name',
       'sport_id','team_id']]
player_df = player_df.dropna()
player_df.loc[player_df['sport_id'] != 'ESPORTS', 'full_name'] = player_df['first_name']+' ' + player_df['last_name']
player_df.loc[player_df['sport_id'] == 'ESPORTS', 'full_name'] = player_df['last_name']
#player_df['full_name'] = player_df['first_name']+' ' + player_df['last_name']
player_df = player_df[['full_name','sport_id','team_id']]
p2l = player_df[['full_name','sport_id']].to_dict('split')
p2tm = player_df[['full_name','team_id']].to_dict('split')
temp = p2l['data']
temp2 = p2tm['data']
p2l = dict(temp)
p2tm = dict(temp2)

In [118]:
player_df

,full_name,sport_id,team_id
0,Joel Embiid,NBA,4dd50e92-0a87-4a6a-bb5d-31c36d88ae72
1,Tyrese Maxey,NBA,4dd50e92-0a87-4a6a-bb5d-31c36d88ae72
2,Tobias Harris,NBA,4dd50e92-0a87-4a6a-bb5d-31c36d88ae72
3,Kelly Oubre Jr.,NBA,4dd50e92-0a87-4a6a-bb5d-31c36d88ae72
4,De'Anthony Melton,NBA,4dd50e92-0a87-4a6a-bb5d-31c36d88ae72
...,...,...,...
1004,J. Harden + R. Westbrook,NBACOMBOS,b231abcd-c8eb-4f5c-b5a9-70f3b97c8b3e
1005,K. Hayes + I. Stewart,NBACOMBOS,81961de1-1a9c-4f87-a145-668a926bf026
1006,P. Washington + M. Williams,NBACOMBOS,cb8b8955-a786-4bb3-8fd0-5b438e61d3d1
1007,T. Harris + D. Melton,NBACOMBOS,19ecbc41-1201-478c-9937-c339c3963b69


In [119]:
player_df.sport_id.unique()

array(['NBA', 'NFL', 'CFB', 'CBB', 'NHL', 'FIFA', 'ESPORTS', 'BASKETBALL',
       'CFL', 'NFLCOMBO', 'NBACOMBOS', 'CFBCOMBOS'], dtype=object)

In [120]:
first2 = player_df['full_name'].apply(lambda x: ' '.join(x.split()[0:2]))
names = pd.DataFrame((first2,player_df.full_name))
names = names.T
names.columns=['first2','fullname']
names.loc[names.first2=='Jazz Chisholm']
first2full = names.to_dict('split')
first2full = dict(first2full['data'])

In [121]:
games = pd.json_normalize(df.T['games'].values).dropna(how='all')
#games = games.loc[games['sport_id']=='NBA']
games.tail(3)

,id,away_team_id,away_team_score,home_team_id,home_team_score,match_progress,period,scheduled_at,season_type,sport_id,status,title,type,year
96,57742.0,7a9d17b6-38c3-402a-9f78-4626e1db104d,0.0,fd67152d-cfb5-47d5-8113-6a37da8a73da,0.0,Fri 03:00pm,0.0,2023-11-10T20:00:00Z,regular,BASKETBALL,scheduled,Lyon-Villeurbanne @ Monaco,Game,2023.0
97,57684.0,cb31c28e-6b10-459b-a8e8-123f37715b0d,0.0,2822c78a-243c-4365-ab0f-bfef88cbac45,0.0,Sat 03:00pm,0.0,2023-11-11T20:00:00Z,regular,CFL,scheduled,MTL @ TOR,Game,2023.0
98,57685.0,c3cb471d-2cbd-4cb4-9d57-c8ef7b7d380b,0.0,643ebde6-4db7-48c1-844d-8aa5f8482746,0.0,Sat 06:30pm,0.0,2023-11-11T23:30:00Z,regular,CFL,scheduled,BC @ WPG,Game,2023.0


In [122]:
def to_central(date_str):
    x = pd.to_datetime(date_str)
    x = x.tz_convert('US/Central')
    return str(x.date())
to_central('2023-05-20T00:40:00Z')

'2023-05-19'

In [123]:
games.scheduled_at = games.scheduled_at.apply(to_central)

In [124]:
#grabbing team names from match title
matchup = games['title']
home = []
away = []
for i in matchup:
    temp = re.split(' @ |, |vs.',i)
    home.append(temp[0])
    away.append(temp[1])
#fixing esports team names
for i in range(len(home)):
    home[i] = home[i].split(': ')[-1]
    
for i in range(len(away)):
    away[i] = away[i].split(': ')[-1]

matchup = pd.DataFrame((away,home,games.away_team_id,games.home_team_id)).T
matchup.columns = ['Home', 'Away','away_id','home_id']
dates = games['scheduled_at'].values
#temp = []
#for i in dates:
#    temp.append(i.split('T')[0])
matchup['Date'] = dates
id2tm =  matchup[['home_id','Home']].to_dict('split')
id2tmaway =  matchup[['away_id','Away']].to_dict('split')
id2tm = dict(id2tm['data'])
id2tmaway = dict(id2tmaway['data'])
id2tm.update(id2tmaway)

In [125]:
games

,id,away_team_id,away_team_score,home_team_id,home_team_score,match_progress,period,scheduled_at,season_type,sport_id,status,title,type,year
0,52476.0,4dd50e92-0a87-4a6a-bb5d-31c36d88ae72,0.0,47f5f48d-3d14-46f3-b6dc-1e7b4dfb1e69,0.0,Fri 07:00pm,0.0,2023-11-10,regular,NBA,scheduled,PHI @ DET,Game,2023.0
1,52475.0,11cfe154-8ba6-4c22-be8f-2365656fb4da,0.0,402685bd-a73c-4eb1-8c0f-55debd622215,0.0,Fri 07:00pm,0.0,2023-11-10,regular,NBA,scheduled,CHA @ WAS,Game,2023.0
2,52477.0,a34e5fa9-e4aa-4195-98bf-05710d97347d,0.0,8f5852b6-c8dc-4cce-980a-eab09b28919a,0.0,Fri 07:30pm,0.0,2023-11-10,regular,NBA,scheduled,BKN @ BOS,Game,2023.0
3,52478.0,f15a9fdd-f178-42ff-aab6-63b9198a34c8,0.0,fee4a894-1970-402a-873b-8faea88f4ec9,0.0,Fri 08:00pm,0.0,2023-11-10,regular,NBA,scheduled,MIN @ SAS,Game,2023.0
4,52480.0,ade6b8fc-178f-4e7e-b43d-aa853bea0f3a,0.0,56a5d8c8-eb72-47db-9e93-5b24d8f5443a,0.0,Fri 08:00pm,0.0,2023-11-10,regular,NBA,scheduled,UTA @ MEM,Game,2023.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,57740.0,4bb9808b-d679-4a63-9755-f1e89861e726,0.0,889a9c83-01b9-4ee9-a00b-04c7004c2033,0.0,Fri 02:00pm,0.0,2023-11-10,regular,BASKETBALL,scheduled,Panathinaikos @ Alba Berlin,Game,2023.0
95,57741.0,cccce565-c662-4118-90d8-700bd3da5b70,0.0,c2392e23-8b66-4062-8860-48021a586dc8,0.0,Fri 02:30pm,0.0,2023-11-10,regular,BASKETBALL,scheduled,Crvena Zvezda @ FC Barcelona,Game,2023.0
96,57742.0,7a9d17b6-38c3-402a-9f78-4626e1db104d,0.0,fd67152d-cfb5-47d5-8113-6a37da8a73da,0.0,Fri 03:00pm,0.0,2023-11-10,regular,BASKETBALL,scheduled,Lyon-Villeurbanne @ Monaco,Game,2023.0
97,57684.0,cb31c28e-6b10-459b-a8e8-123f37715b0d,0.0,2822c78a-243c-4365-ab0f-bfef88cbac45,0.0,Sat 03:00pm,0.0,2023-11-11,regular,CFL,scheduled,MTL @ TOR,Game,2023.0


In [126]:
id2tm

{'47f5f48d-3d14-46f3-b6dc-1e7b4dfb1e69': 'DET',
 '402685bd-a73c-4eb1-8c0f-55debd622215': 'WAS',
 '8f5852b6-c8dc-4cce-980a-eab09b28919a': 'BOS',
 'fee4a894-1970-402a-873b-8faea88f4ec9': 'SAS',
 '56a5d8c8-eb72-47db-9e93-5b24d8f5443a': 'MEM',
 '2c204562-627d-4fbe-b9ba-24a8bbf87463': 'DAL',
 'f98f3baa-ad03-4188-9e8a-0cb52e1b900e': 'PHX',
 'f97fdf6e-8328-4000-ba49-0b2ce0beae71': 'SAC',
 'ef876de9-81ac-5e60-af6e-0ca1700f3a51': 'NE',
 'b49b653c-dc7c-516e-b1c2-da1b30d1b1a6': 'JAX',
 '01bfe9d5-f671-57ed-aa60-249fcca9267c': 'BAL',
 'ab235cf0-a041-5d36-8241-90a90f0dcb5e': 'CIN',
 '5af1e185-627e-5345-851d-3c65c5b66614': 'MIN',
 'ecc8eb1b-f714-57a6-bcf3-b183dd6c12a8': 'PIT',
 '1a20f1da-c502-5224-9c4a-bc363174cd21': 'TB',
 'c7b497d4-18b6-522b-abaa-e5c3d24bc021': 'LAC',
 '8459772c-695a-5890-afa1-7ec38da17201': 'ARI',
 '31631011-5902-52f6-ba01-c4c8d8eb3fd9': 'SEA',
 '26e67e06-664e-50a6-ad7b-c102705fde8b': 'DAL',
 '5ce78c37-b02c-52da-9e4f-fcd65b6ccb76': 'LV',
 '0719b253-43db-532e-8b5c-6c12c6c3f951': 'B

In [127]:
lines = lines[['stat_value','over_under.appearance_stat.stat', 'over_under.title']]
lines
players = lines['over_under.title'].values
guys = []
for i in players:
    temp = i.split()
    final = temp[0] + ' '+temp[1]
    guys.append(final)
    
guys
lines= lines.rename(columns={'stat_value':'Line','over_under.appearance_stat.stat':'Stat', 'over_under.title':'title'})
lines['Player']= guys
lines['Player'] = lines['Player'].apply(lambda x: unidecode(x).replace('_',' '))


In [128]:
def map_name(x):
    if x is None:
        return None
    if x in first2full:
        return first2full[x]
    elif x.split(': ')[-1] in first2full:
        return x.split(': ')[-1]
    elif x.split()[-1] in first2full:
        return x.split()[-1]
    else:
        return x

lines['Player'] = lines['Player'].apply(lambda x: map_name(x))
lines.Player.unique()

array(['Joel Embiid', 'Tyrese Maxey', 'Tobias Harris', ..., 'J. Milton',
       'J. Wright', 'S. White + R. Keyton'], dtype=object)

In [129]:
league = []
for p in guys:
    if p in p2l:
        league.append(p2l[p])
    elif p.split()[-1] in p2l:
        league.append(p2l[p.split()[-1]])
    else:
        league.append('other')
league[0:5]

['NBA', 'NBA', 'NBA', 'NBA', 'NBA']

In [130]:
guys[1000:1007]

['Jared Goff',
 'Jared Goff',
 'Jared Goff',
 'Jared Goff',
 'Amon-Ra St.',
 'Amon-Ra St.',
 'Amon-Ra St.']

In [131]:
team_id = []
for p in guys:
    if p in p2l:
        team_id.append(p2tm[p])
    elif p.split()[-1] in p2l:
        team_id.append(p2tm[p.split()[-1]])
    else:
        team_id.append('other')
team_id[0:5]

['4dd50e92-0a87-4a6a-bb5d-31c36d88ae72',
 '4dd50e92-0a87-4a6a-bb5d-31c36d88ae72',
 '4dd50e92-0a87-4a6a-bb5d-31c36d88ae72',
 '4dd50e92-0a87-4a6a-bb5d-31c36d88ae72',
 '4dd50e92-0a87-4a6a-bb5d-31c36d88ae72']

In [132]:
lines['League'] = league
lines

,Line,Stat,title,Player,League
0,29.5,points,Joel Embiid Points O/U,Joel Embiid,NBA
1,45.5,pts_rebs_asts,Joel Embiid Pts + Rebs + Asts O/U,Joel Embiid,NBA
2,4.5,assists,Joel Embiid Assists O/U,Joel Embiid,NBA
3,10.5,rebounds,Joel Embiid Rebounds O/U,Joel Embiid,NBA
4,15.5,rebs_asts,Joel Embiid Rebounds + Assists O/U,Joel Embiid,NBA
...,...,...,...,...,...
3355,41.5,pts_rebs_asts,T. Harris + D. Melton Pts + Rebs + Asts O/U,T. Harris + D. Melton,other
3356,3.5,passing_tds,J. Milton III + B. Cook Passing TDs O/U,J. Milton,other
3357,510.5,passing_yds,J. Milton III + B. Cook Passing Yards O/U,J. Milton,other
3358,164.5,rushing_yds,J. Wright + C. Schrader Rushing Yards O/U,J. Wright,other


In [133]:
def map_team(x):
    if x in p2tm:
        return p2tm[x]
    elif p.split()[-1] in p2l:
        return p2tm[p.split()[-1]]
    else:
        return x

lines['team_id'] = lines.Player.apply(lambda x: map_team(x))
lines

,Line,Stat,title,Player,League,team_id
0,29.5,points,Joel Embiid Points O/U,Joel Embiid,NBA,4dd50e92-0a87-4a6a-bb5d-31c36d88ae72
1,45.5,pts_rebs_asts,Joel Embiid Pts + Rebs + Asts O/U,Joel Embiid,NBA,4dd50e92-0a87-4a6a-bb5d-31c36d88ae72
2,4.5,assists,Joel Embiid Assists O/U,Joel Embiid,NBA,4dd50e92-0a87-4a6a-bb5d-31c36d88ae72
3,10.5,rebounds,Joel Embiid Rebounds O/U,Joel Embiid,NBA,4dd50e92-0a87-4a6a-bb5d-31c36d88ae72
4,15.5,rebs_asts,Joel Embiid Rebounds + Assists O/U,Joel Embiid,NBA,4dd50e92-0a87-4a6a-bb5d-31c36d88ae72
...,...,...,...,...,...,...
3355,41.5,pts_rebs_asts,T. Harris + D. Melton Pts + Rebs + Asts O/U,T. Harris + D. Melton,other,19ecbc41-1201-478c-9937-c339c3963b69
3356,3.5,passing_tds,J. Milton III + B. Cook Passing TDs O/U,J. Milton,other,J. Milton
3357,510.5,passing_yds,J. Milton III + B. Cook Passing Yards O/U,J. Milton,other,J. Milton
3358,164.5,rushing_yds,J. Wright + C. Schrader Rushing Yards O/U,J. Wright,other,J. Wright


In [134]:
def map_id(x):
    if x in id2tm:
        return id2tm[x]
    else:
        return x
#lines['team_id'] = lines.team_id.apply(lambda x: map_id(x))


In [135]:
final = pd.merge(lines,matchup, how='left',left_on='team_id',right_on='away_id')
final = final.drop(['Home', 'Away','home_id','away_id'], axis=1)
final2 = pd.merge(final,matchup, how='left',left_on='team_id',right_on='home_id')
final2 = final2.drop(['Home', 'Away','home_id','away_id'], axis=1)
final2['Date'] = final2['Date_x'].fillna(final2['Date_y'])
final2 = final2.drop(['Date_x', 'Date_y'], axis=1)
final2['team_id'] = final2.team_id.apply(lambda x: map_id(x))
final2#.loc[final.Player=='Jimmy Butler']

,Line,Stat,title,Player,League,team_id,Date
0,29.5,points,Joel Embiid Points O/U,Joel Embiid,NBA,PHI,2023-11-10
1,45.5,pts_rebs_asts,Joel Embiid Pts + Rebs + Asts O/U,Joel Embiid,NBA,PHI,2023-11-10
2,4.5,assists,Joel Embiid Assists O/U,Joel Embiid,NBA,PHI,2023-11-10
3,10.5,rebounds,Joel Embiid Rebounds O/U,Joel Embiid,NBA,PHI,2023-11-10
4,15.5,rebs_asts,Joel Embiid Rebounds + Assists O/U,Joel Embiid,NBA,PHI,2023-11-10
...,...,...,...,...,...,...,...
3355,41.5,pts_rebs_asts,T. Harris + D. Melton Pts + Rebs + Asts O/U,T. Harris + D. Melton,other,19ecbc41-1201-478c-9937-c339c3963b69,NaN
3356,3.5,passing_tds,J. Milton III + B. Cook Passing TDs O/U,J. Milton,other,J. Milton,NaN
3357,510.5,passing_yds,J. Milton III + B. Cook Passing Yards O/U,J. Milton,other,J. Milton,NaN
3358,164.5,rushing_yds,J. Wright + C. Schrader Rushing Yards O/U,J. Wright,other,J. Wright,NaN


In [136]:
final2.columns = ['Line', 'Stat', 'title', 'Player', 'League', 'Team', 'Date']

In [137]:
df = final2[['Line', 'Stat', 'Player', 'League', 'Team', 'Date']].copy(deep=True)
df

,Line,Stat,Player,League,Team,Date
0,29.5,points,Joel Embiid,NBA,PHI,2023-11-10
1,45.5,pts_rebs_asts,Joel Embiid,NBA,PHI,2023-11-10
2,4.5,assists,Joel Embiid,NBA,PHI,2023-11-10
3,10.5,rebounds,Joel Embiid,NBA,PHI,2023-11-10
4,15.5,rebs_asts,Joel Embiid,NBA,PHI,2023-11-10
...,...,...,...,...,...,...
3355,41.5,pts_rebs_asts,T. Harris + D. Melton,other,19ecbc41-1201-478c-9937-c339c3963b69,NaN
3356,3.5,passing_tds,J. Milton,other,J. Milton,NaN
3357,510.5,passing_yds,J. Milton,other,J. Milton,NaN
3358,164.5,rushing_yds,J. Wright,other,J. Wright,NaN


In [138]:
df = df.reset_index(drop=True)

In [139]:
df.Stat.unique()

array(['points', 'pts_rebs_asts', 'assists', 'rebounds', 'rebs_asts',
       'pts_asts', 'pts_rebs', 'field_goals_att', 'three_points_att',
       'blocks', 'blks_stls', 'three_points_made', 'personal_fouls',
       'turnovers', 'steals', 'double_doubles', 'passing_yds',
       'rushing_yds', 'passing_att', 'passing_comps', 'passing_ints',
       'passing_and_rushing_yds', 'rushing_att', 'receiving_yds',
       'rush_rec_yds', 'receiving_rec', 'field_goals_made',
       'kicking_points', 'tackles_and_assists', 'tackles', 'passing_tds',
       'fantasy_points', 'sacks', 'rush_rec_tds', 'extra_points_made',
       'shots', 'blocked_shots', 'goals', 'saves', 'shots_attempted',
       'goals_assists', 'shots_on_target', 'goals_against',
       'fouls_against', 'aces', 'double_faults', 'games_won',
       'games_lost', 'games_played', 'strokes', 'kills_in_map_1',
       'kills_in_maps_1_2_3', 'kills_in_map_3', 'kills_in_map_2',
       'fantasy_points_on_map_1', 'fantasy_points_on_maps_1_2_3

In [140]:
syntax = {
    'pts_rebs_asts': 'PTS+TRB+AST',
    'pts_rebs': 'PTS+TRB',
    'points': 'PTS',
    'free_throws_made': 'FT',
    'fantasy_points': 'Fantasy',
    'rebounds': 'TRB',
    'assists': 'AST',
    'steals': 'STL',
    'blocks': 'BLK',
    'blks_stls': 'BLK+STL',
    'turnovers': 'TO',
    'rebs_asts': 'TRB+AST',
    'pts_asts': 'PTS+AST',
    'three_points_made': '3PM',
    'shots': 'Shots',
    'goals': 'Goals',
    'saves': 'Goalie Saves',
    'clearances': 'Clearances',
    'goals_and_assists': 'Goals and Assists',
    'tackles': 'Tackles',
    'interceptions': 'Interceptions',
    'crosses': 'Crosses',
    'goals_against': 'Goal Against',
    'game_1_and_2_kills': 'MAPS 1-2 Kills',
    'map_1_and_2_kills': 'MAPS 1-2 Kills',
    'kills_in_maps_1_2': 'MAPS 1-2 Kills',
    'Headshots on Maps 1+2': 'MAPS 1-2 Headshots',
    'strokes': 'Strokes',
    'bogeys_or_worse': 'Bogeys or Worse',
    'birdies_or_better': 'Birdies or Better',
    'aces': 'Aces',
    'breakpoints_won': 'Break Points Won',
    'games_won': 'Games Won',
    'games_lost': 'Games Lost',
    'total_games': 'Total Games',
    'double_faults': 'Double Faults',
    'service_games_lost': 'Service Games Lost',
    'Fastest Laps': 'Fastest Laps',
    'position': 'position',
    'passing_yds': 'Pass Yards',
    'rushing_yds': 'Rush Yards',
    'receiving_yds': 'Receiving Yards' ,
    'fantasy_points': 'Hitter Fantasy Score',
    'strikeouts': 'Pitcher Strikeouts',
    'hits_runs_rbis': 'PTS',
    'singles':'Singles',
    'total_bases': 'Total Bases',
    'walks': 'Walks Allowed',
    'hits': 'Hits Allowed',
    'kills_in_map_1' : 'MAPS 1 Kills',
    'kills_in_maps_1_2_3' : 'MAPS 1-3 Kills',
    'kills_in_map_1': 'MAP 1 Kills',
    'kills_in_map_2': 'MAP 2 Kills',
    'kills_in_map_3': 'MAP 3 Kills',
    'fantasy_points_on_maps_1_2_3': 'MAPS 1-3 Fantasy points',
    'fantasy_points_on_map_1': 'MAP 1 Fantasy points',
    'fantasy_points_on_map_2': 'MAP 2 Fantasy points',
    'fantasy_points_on_map_3': 'MAP 3 Fantasy points',
    'deaths_on_maps_1_2_3' : 'MAP 1-3 Deaths',
    'deaths_in_map_2' : 'MAP 2 Deaths',
    'deaths_in_map_3' : 'MAP 3 Deaths',
    'field_goals_att': 'FGA',
    'three_points_att': '3PA',
    'personal_fouls': 'PF',
    'passing_att': 'Pass Attempts',
    'passing_comps': 'Pass Completions',
    'passing_ints': 'Pass Interceptions',
    'passing_and_rushing_yds': 'Pass and Rush Yards',
    'rushing_att': 'Rush Attempts',
    'rush_rec_yds': 'Rush+Rec Yards',
    'receiving_rec': 'Receptions',
    'field_goals_made': 'FGM',
    'kicking_points': 'Kicking Points',
    'tackles_and_assists': 'Tackles+Ast',
    'passing_tds': 'Pass TD',
    'rush_rec_tds': 'Rush+Rec Touchdowns',
    'extra_points_made': 'XP',
    'blocked_shots': 'BLK',
    'shots_attempted': 'Shots',
    'goals_assists': 'Goals and Assists',
    'shots_on_target': 'Shots on Target',
    'fouls_against': 'Fouls Against',
    'assists_on_map_1': 'MAP 1 Assists',
    'assists_in_map_2': 'MAP 2 Assists',
    'fantasy_points_in_map_3': 'MAP 3 Fantasy Points',
    'assists_on_maps_1_2_3': 'MAP 1-3 Assists',
    'assists_in_map_3': 'MAP 3 Assists',
    'kills_on_maps_1_2_3': 'MAP 1-3 Kills',
    'deaths_on_map_1': 'MAP 1 Deaths',
    'headshots_on_maps_1_2': 'MAP 1-2 Headshots',
    'significant_strikes': 'Significant Strikes',
    'fight_time': 'Fight Time',
    'total_yds': 'Total Yards',
    'games_played': 'GP',
                }

In [141]:
df = df.replace(syntax)

In [142]:
df.loc[df['League'] == 'ESPORTS']

,Line,Stat,Player,League,Team,Date
2138,5.5,MAP 1 Kills,Bin+XUN,ESPORTS,Bilibili Gaming,2023-11-11
2139,6.5,MAP 1 Kills,Elk+Yagao,ESPORTS,Bilibili Gaming,2023-11-11
2140,17.0,MAPS 1-3 Kills,Bin+XUN,ESPORTS,Bilibili Gaming,2023-11-11
2141,19.5,MAPS 1-3 Kills,Elk+Yagao,ESPORTS,Bilibili Gaming,2023-11-11
2142,6.5,MAP 3 Kills,Elk+Yagao,ESPORTS,Bilibili Gaming,2023-11-11
...,...,...,...,...,...,...
3165,10.0,MAPS 1-2 Kills,pr1me,ESPORTS,Orbit Anonymo,2023-11-10
3166,8.5,MAPS 1-2 Kills,Savero,ESPORTS,Orbit Anonymo,2023-11-10
3167,10.5,MAPS 1-2 Kills,Roison,ESPORTS,Orbit Anonymo,2023-11-10
3168,2.0,MAPS 1-2 Kills,Czypsy,ESPORTS,Orbit Anonymo,2023-11-10


In [143]:
df.loc[df.Player=='Krimbo']

,Line,Stat,Player,League,Team,Date


In [144]:
today = dt.date.today()

In [145]:
str(today)

'2023-11-10'

In [146]:
df.columns = [x.lower() for x in df.columns]
df.to_csv(f'Lines/dog/doglines_{today.month}_{today.day}.csv')

In [147]:
df.loc[df.league=='ESPORTS']

,line,stat,player,league,team,date
2138,5.5,MAP 1 Kills,Bin+XUN,ESPORTS,Bilibili Gaming,2023-11-11
2139,6.5,MAP 1 Kills,Elk+Yagao,ESPORTS,Bilibili Gaming,2023-11-11
2140,17.0,MAPS 1-3 Kills,Bin+XUN,ESPORTS,Bilibili Gaming,2023-11-11
2141,19.5,MAPS 1-3 Kills,Elk+Yagao,ESPORTS,Bilibili Gaming,2023-11-11
2142,6.5,MAP 3 Kills,Elk+Yagao,ESPORTS,Bilibili Gaming,2023-11-11
...,...,...,...,...,...,...
3165,10.0,MAPS 1-2 Kills,pr1me,ESPORTS,Orbit Anonymo,2023-11-10
3166,8.5,MAPS 1-2 Kills,Savero,ESPORTS,Orbit Anonymo,2023-11-10
3167,10.5,MAPS 1-2 Kills,Roison,ESPORTS,Orbit Anonymo,2023-11-10
3168,2.0,MAPS 1-2 Kills,Czypsy,ESPORTS,Orbit Anonymo,2023-11-10


In [148]:
df.stat.unique()

array(['PTS', 'PTS+TRB+AST', 'AST', 'TRB', 'TRB+AST', 'PTS+AST',
       'PTS+TRB', 'FGA', '3PA', 'BLK', 'BLK+STL', '3PM', 'PF', 'TO',
       'STL', 'double_doubles', 'Pass Yards', 'Rush Yards',
       'Pass Attempts', 'Pass Completions', 'Pass Interceptions',
       'Pass and Rush Yards', 'Rush Attempts', 'Receiving Yards',
       'Rush+Rec Yards', 'Receptions', 'FGM', 'Kicking Points',
       'Tackles+Ast', 'Tackles', 'Pass TD', 'Hitter Fantasy Score',
       'sacks', 'Rush+Rec Touchdowns', 'XP', 'Shots', 'Goals',
       'Goalie Saves', 'Goals and Assists', 'Shots on Target',
       'Goal Against', 'Fouls Against', 'Aces', 'Double Faults',
       'Games Won', 'Games Lost', 'games_played', 'Strokes',
       'MAP 1 Kills', 'MAPS 1-3 Kills', 'MAP 3 Kills', 'MAP 2 Kills',
       'MAP 1 Fantasy points', 'MAPS 1-3 Fantasy points', 'MAP 3 Assists',
       'MAP 3 Deaths', 'MAP 2 Fantasy points', 'MAP 1-3 Deaths',
       'MAP 1-3 Assists', 'MAP 3 Fantasy Points', 'MAP 2 Assists',
       'MAP 1